In [2]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy
import pandas as pd

Using TensorFlow backend.


In [3]:
dataset = pd.read_csv('pima-indians-diabetes.csv')
dataset.head(3)

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0


In [4]:
# split into input(X) and output(Y) variables
X = dataset.iloc[:,:-1].values
Y = dataset.iloc[:,-1].values

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12,input_dim = 8, init = 'uniform', activation = 'relu'))
    model.add(Dense(8,init = 'uniform', activation = 'relu'))
    model.add(Dense(1, init = 'uniform', activation = 'sigmoid'))
    # compile model
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn = create_model, nb_epoch = 150, batch_size = 10, verbose = 0)

# evaluate using 10-fold cross validation
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = seed)
results = cross_val_score(model,X,Y, cv = kfold)
print(results.mean())

C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="uniform")`
  del sys.path[0]
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
  
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  from ipykernel import kernelapp as app


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


0.651913869380951


- Running the example displays the skill of the model for each epoch. A total of 10 models are created and evaluated and the final average accuracy is displayed.

# Grid Search Deep Learning Model Parameters
- The previous example showed how easy it is to wrap your deep learning model from Keras and use it in functions from the scikit-learn library. In this example we go a step further. We already know we can provide arguments to the fit() function. The function that we specify to the build fn argument when creating the KerasClassifier wrapper can also take arguments. We can use these arguments to further customize the construction of the model.
- In this example we use a grid search to evaluate di↵erent configurations for our neural network model and report on the combination that provides the best estimated performance. The create_model() function is defined to take two arguments optimizer and init, both of which must have default values. This will allow us to evaluate the e↵ect of using di↵erent optimization algorithms and weight initialization schemes for our network. After creating our model, we define arrays of values for the parameter we wish to search, specifically:
    - Optimizers for searching di↵erent weight values.
    - Initializers for preparing the network weights using different schemes.
    - Number of epochs for training the model for di↵erent number of exposures to the training dataset.
    - Batches for varying the number of samples before weight updates.
- The options are specified into a dictionary and passed to the configuration of the GridSearchCV scikit-learn class. This class will evaluate a version of our neural network model for each combination of parameters (2 X 3 X 3 X 3) for the combinations of optimizers, initializations, epochs and batches). Each combination is then evaluated using the default of 3-fold stratified cross validation.
- That is a lot of models and a lot of computation. This is not a scheme that you want to use lightly because of the time it will take to compute. It may be useful for you to design small experiments with a smaller subset of your data that will complete in a reasonable time. This experiment is reasonable in this case because of the small network and the small dataset (less than 1,000 instances and 9 attributes). Finally, the performance and combination of configurations for the best model are displayed, followed by the performance of all combinations of parameters.

In [5]:
# MLP for Pima Indians Dataset with grid search via sklearn

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import numpy
import pandas as pd

In [6]:
dataset = pd.read_csv('pima-indians-diabetes.csv')
dataset.head(3)

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0


In [12]:
# split into input(X) and output (Y) variables
X = dataset.iloc[:,:-1].values
Y = dataset.iloc[:,-1].values

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Function to create model, required for KerasClassifier
def create_model(optimizer = 'rmsprop', init = 'glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim = 8, init = init, activation = 'relu'))
    model.add(Dense(8, init = init, activation = 'relu'))
    model.add(Dense(1, init = init, activation = 'sigmoid'))
    # compile model
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn = create_model, verbose = 0)

# grid search epochs, batch size and optimizer
optimizer = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal','uniform']
epochs = numpy.array([50,100,150])
batches = numpy.array([5,10,20])
param_grid = dict(optimizer = optimizer, nb_epoch = epochs, batch_size = batches, init = init)
grid = GridSearchCV(estimator = model,param_grid = param_grid)
grid_result = grid.fit(X,Y)

# summarize results
print("Best : %f using %s" % (grid_result.best_score_, grid_result.best_params_))

C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="glorot_uniform")`
  del sys.path[0]
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="glorot_uniform")`
  
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform")`
  from ipykernel import kernelapp as app
C:\Users\alekh\AppData\Local\conda\conda\envs\Tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="normal")`
  del sys.p

Best : 0.655802 using {'batch_size': 5, 'init': 'normal', 'nb_epoch': 100, 'optimizer': 'rmsprop'}


In [14]:
#grid_result.grid_score_